# Nosana: WAN Video Generation Tutorial

In this tutorial, we will use a Jupyter Notebook kernel URL hosted on the [Nosana](!https://dashboard.nosana.com/) GPU network. This allows you to access powerful GPU resources on demand, making it easy to run video generation workloads without local hardware limitations.

In this notebook, we will focus on the **Text-to-Video** capabilities of the `Wan2.1-T2V-1.3B` model, a version optimized for consumer-grade GPUs. We will cover the essential concepts to get you started on your video generation journey.

### Key Concepts We'll Cover:
* **Setting Up the Environment**: Installing necessary libraries and loading API keys.
* **Loading the Model**: How to load the Wan 2.1 pipeline using the `diffusers` library.
* **Text-to-Video Generation**: Creating your first AI-generated video from a text prompt.
* **Core Parameters**: Understanding and using parameters like `prompt`, `negative_prompt`, `num_frames`, and `guidance_scale` to control the output.
* **Saving Your Video**: Exporting the generated frames into a video file.

Let's get started!

## 1. Setup and Installation

First, let's install the required Python libraries. We need `diffusers` for the video generation pipeline, `transformers` for model components, `torch` for the deep learning framework, `python-dotenv` to manage our API keys, and `imageio` to save the video file.

In [3]:
# !pip install diffusers transformers torch python-dotenv imageio -q

In [ ]:
# Install ipywidgets for tqdm progress bars and accelerate for efficient model loading
# !pip install ipywidgets accelerate --upgrade -q

### Loading API Keys from `.env` File

It's a best practice to store sensitive information like API keys in a `.env` file. To access models from the Hugging Face Hub, you'll need a User Access Token. 

1.  Create a file named `.env` in the same directory as this notebook.
2.  Add your Hugging Face token to the file like this:
    ```
    HUGGINGFACE_ACCESS_TOKEN="hf_your_token_here"
    ```
3.  The code below will load this key into our environment.

In [10]:
#!echo "HUGGINGFACE_ACCESS_TOKEN=hf_your_token_here" > .env

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the token is loaded
hf_token = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
if hf_token:
    print("Hugging Face token loaded successfully!")
else:
    print("Hugging Face token not found. Please check your .env file.")

Hugging Face token loaded successfully!


## 2. Loading the Wan 2.1 Pipeline

Now we'll load the Wan 2.1 model using the `WanPipeline` from the `diffusers` library. We'll use the `Wan-AI/Wan2.1-T2V-1.3B-Diffusers` model, which is a great starting point. We also specify the data type (`torch.float16`) to ensure it runs efficiently on GPUs.

In [ ]:
import torch
from diffusers import WanPipeline
from diffusers.utils import export_to_video

# Define the model ID and device
model_id = "Wan-AI/Wan2.1-T2V-1.3B-Diffusers"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

# Load the pipeline
pipe = WanPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_auth_token=hf_token,
    low_cpu_mem_usage=True
).to(device)

print("Wan 2.1 pipeline loaded successfully!")

Using device: cpu


model_index.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00002.safetensors:   0%|          | 0.00/677M [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/508M [00:00<?, ?B/s]

## 3. Generating Your First Video

With the pipeline loaded, we can now generate a video. This involves defining a **prompt** to describe the scene you want to create. You can also use a **negative prompt** to specify things you want to avoid in the video, which helps improve quality.

### Key Generation Parameters:
- **`prompt`**: A string describing the desired video content.
- **`negative_prompt`**: A string describing what to avoid (e.g., 'low quality, blurry').
- **`num_frames`**: The total number of frames in the video. A higher number means a longer video.
- **`guidance_scale`**: How strongly the model should follow the prompt. A value between `5.0` and `7.5` is usually a good starting point.
- **`height` & `width`**: The resolution of the video. The 1.3B model is optimized for resolutions around 480p.

In [ ]:
# Define the prompt and negative prompt
prompt = "a majestic lion shaking its mane in slow motion, cinematic lighting"
negative_prompt = "blurry, low quality, cartoon, watermark, text"

# Generate the video frames
video_frames = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_frames=40, # Approx. 2.5 seconds at 16 FPS
    guidance_scale=7.0,
    height=480,
    width=832
).frames[0]

print("Video frames generated successfully!")

## 4. Saving the Video

The pipeline returns a list of frames. We can use the `export_to_video` utility function from `diffusers` to easily save these frames as an MP4 file.

In [ ]:
# Define the output path and frames per second (fps)
output_video_path = "lion_video.mp4"
fps = 16

# Export the frames to a video file
export_to_video(video_frames, output_video_path, fps=fps)

print(f"Video saved to: {output_video_path}")

## Conclusion and Next Steps

Congratulations! You've successfully generated your first video using Wan 2.1. 

This tutorial covered the basics of setting up your environment, loading the model, and generating a video from a text prompt. From here, you can:

* **Experiment with different prompts**: Try more complex and creative descriptions.
* **Adjust parameters**: See how changing `guidance_scale` and `num_frames` affects the output.
* **Explore other models**: Try larger models like `Wan2.1-T2V-14B` for higher resolution if you have the hardware.
* **Learn about Image-to-Video**: Explore how to animate still images.
* **Fine-tuning**: For advanced use cases, you can fine-tune Wan 2.1 on your own datasets to create specialized video models.

The field of AI video generation is evolving rapidly. Keep exploring, and have fun creating!